In [1]:
import os
import copy
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt 

from dekef.base_density import *

from IFdensity.contam_sm_de import *
from IFdensity.influence_function import *

from IPython.display import Markdown as md

In [2]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFdensity')
true_data = np.load('data/geyser.npy').astype(np.float64)
df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)

# original data with 108.0 removed 
data_waiting = df[df != 108.0]

# bandwidth parameter in the Gaussian kernel function
bw = 7.0

# penalty parameter 
log_pen_param = -10.0

# base density 
base_density = BasedenGamma(np.load('data/geyser.npy').astype(np.float64)[:, 0])

plot_kwargs = plot_IF_1d_params(x_limit=(21., 410.), plot_pts_cnt = 2000)

In [ ]:
# contaminated data 
contam_data_list = np.linspace(1e20, 1e21, 10)
if_norm = []
if_limit_norm = []
for contam_pt in contam_data_list: 
    
    print('-' * 50)
    print(f'Current contaminated data point is {contam_pt}.')
    
    contam_data = np.array([[contam_pt]])
    
    ifun = SMInfluenceFunction(
        data = data_waiting, 
        contam_data = contam_data, 
        contam_weight = 1e-8, 
        penalty_param = np.exp(log_pen_param), 
        base_density = base_density, 
        bw = bw)
    
#     ifun1 = ifun.plot_IF_logdensity_1d(plot_kwargs, 'waiting')
#     ifun2 = ifun.plot_IF_natparam_1d(plot_kwargs, 'waiting')
#     ifun3 = ifun.plot_IF_natparam_limit_1d(plot_kwargs, 'waiting')
    
    if_norm.append(ifun.eval_IF_natparam_norm())

    if_limit_norm.append(ifun.eval_IF_natparam_limit_norm_1d())

plt.plot(contam_data_list, if_norm, 'b-')
plt.plot(contam_data_list, if_limit_norm, 'r-')

--------------------------------------------------
Current contaminated data point is 1e+20.
4.5399929762484854e-05
K11 = (298, 298), K12 = (298, 1), K13 = (298, 1), K14 = (298, 1)
K21 = (1, 298), K23 = (1, 1).
coef1_1, (298, 1), coef1_2, (298, 1), coef1_3, (298, 1)
coef1, (298, 1), coef2, (1, 1)
coef, (301, 1)
--------------------------------------------------
Current contaminated data point is 2e+20.


In [8]:
contam_pt = 250.
contam_data = np.array([[contam_pt]])
cde = ContamSMDensityEstimate(
        data = data_waiting, 
        contam_data = contam_data, 
        contam_weight = 1e-8, 
        penalty_param = np.exp(log_pen_param), 
        base_density = base_density, 
        bw = bw)

In [5]:
if_limit_norm[0]

1345.5946779512853

In [15]:
def eval_IF_natparam_limit_norm_1d(contam_density, base_density):

    N, d = contam_density.N, contam_density.d
    assert d == 1, f'The function eval_IF_natparam_limit_norm_1d only works for 1-dimensional data, ' \
                   f'but got {d}-dimensional data.'

    pen_param = contam_density.penalty_param
    K11 = contam_density.matrix_K11()
    K13 = contam_density.matrix_K13()
    K31 = contam_density.matrix_K31()
    K33 = contam_density.matrix_K33()
    K11_inv = np.linalg.inv(K11 + N * pen_param * np.eye(N * d))
    prod1 = np.matmul(K11, K11_inv)
    prod2 = np.matmul(prod1, K13) - 2. * K13
    prod3 = np.matmul(K11_inv, prod2)
    gamma_coef = - prod3 / pen_param
    coef = np.vstack((gamma_coef, -1. / pen_param)).reshape(-1, 1)
    
    large_mat_row1 = np.hstack((K11, K13))
    large_mat_row2 = np.hstack((K31, K33))
    large_mat = np.vstack((large_mat_row1, large_mat_row2))
    
    part1 = np.matmul(coef.T, (np.matmul(large_mat, coef))).item()

    # norm{z_{delta_y}}^2 / pen_param ** 2
    if base_density.name == 'Gamma':
        mu_limit = - 1. / base_density.scale

    if contam_density.kernel_type == 'gaussian_poly2':

        if contam_density.kernel_function_data.r2 != 0.:
            raise ValueError('In order to use the function eval_IF_natparam_limit_norm_1d, must set r2 to be 0.')

        ker11 = contam_density.kernel_function_data.r1 * 1. / contam_density.kernel_function_data.bw ** 2
        ker12 = 0.
        ker22 = contam_density.kernel_function_data.r1 * 3. / contam_density.kernel_function_data.bw ** 4

    elif contam_density.kernel_type == 'rationalquad_poly2':

        if contam_density.kernel_function_data.r2 != 0.:
            raise ValueError('In order to use the function eval_IF_natparam_limit_norm_1d, must set r2 to be 0.')

        ker11 = contam_density.kernel_function_data.r1 * 2. / contam_density.kernel_function_data.bw ** 2
        ker12 = 0.
        ker22 = contam_density.kernel_function_data.r1 * 24. / contam_density.kernel_function_data.bw ** 4

    part3 = (ker22 + 2. * mu_limit * ker12 + mu_limit ** 2 * ker11) / pen_param ** 2


    output = part1 + part3

    return np.sqrt(output)


In [16]:
eval_IF_natparam_limit_norm_1d(cde, base_density)

1345.5946779512851

In [ ]:
z1, z2, z4, z3, cde.n

In [ ]:
kf = GaussianPoly2(data = data_waiting.reshape(-1, 1), r1 = 1., r2 = 0., c = 0., bw = bw)

In [ ]:
kernel_partial_11 = kf.partial_kernel_matrix_11(new_data=contam_data)
data_waiting = data_waiting.reshape(-1, 1)
baseden_partial_data = np.zeros(data_waiting.shape, dtype=np.float64)
for u in range(data_waiting.shape[1]):
    baseden_partial_data[:, u] = base_density.logbaseden_deriv1(
        new_data=data_waiting, j=u).flatten()
baseden_partial_data = baseden_partial_data.reshape(-1, 1)

baseden_partial_contam_data = np.zeros(contam_data.shape, dtype=np.float64)
for u in range(contam_data.shape[1]):
    baseden_partial_contam_data[:, u] = base_density.logbaseden_deriv1(
        new_data=contam_data, j=u).flatten()
baseden_partial_contam_data = baseden_partial_contam_data.reshape(-1, 1)

z4 = np.matmul(baseden_partial_data.T, np.matmul(kernel_partial_11, baseden_partial_contam_data))

In [ ]:
z1 = np.sum(kf.partial_kernel_matrix_22(new_data=contam_data))

In [ ]:
kernel_partial_12 = kf.partial_kernel_matrix_12(new_data=contam_data)
z3 = np.sum(baseden_partial_data * kernel_partial_12)

In [ ]:
kernel_partial_21 = kf.partial_kernel_matrix_21(new_data=contam_data)
z2 = np.sum(np.matmul(kernel_partial_21, baseden_partial_contam_data))

In [ ]:
(z1 + z2 + z3 + z4) /298

In [ ]:
(result341+ result342+ result343+ result344)/298